# Data Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine. 

**Authors:** Johannes Giehl (jfg.eco@cbs.dk), Dana Hentschel (djh.eco@cbs.dk)

## General settings

### Packages:

In [1]:
import pandas as pd
from datetime import timedelta
import papermill as pm
from nord_h2ub_dropdown_functions import *

### Base Parameters:

In [2]:
# Choose the year
year_dropdown = create_dropdown('year')
display(year_dropdown)

Dropdown(description='year:', options=(2018, 2019, 2020, 2021, 2022), value=None)

You selected: 2019


In [3]:
# Set the area in which the facility is located
area_dropdown = create_dropdown('area')
display(area_dropdown)

Dropdown(description='area:', index=1, options=('DE', 'DK1', 'DK2', 'NO2', 'SE3', 'SE4', 'SYSTEM'), value='DK1…

In [4]:
# Choose the product to be produced
product_dropdown = create_dropdown('product')
display(product_dropdown)

Dropdown(description='product:', index=1, options=('ammonia', 'methanol', 'jet_fuel'), value='methanol')

In [5]:
# Select the type of electrolyzer
electrolyzer_type_dropdown = create_dropdown('electrolyzer_type')
display(electrolyzer_type_dropdown)

Dropdown(description='electrolyzer_type:', index=1, options=('PEM', 'Alkaline', 'SOEC'), value='Alkaline')

In [6]:
# Choose the number of segments for the variable efficiency
des_segments_electrolyzer_dropdown = create_dropdown('des_segments_electrolyzer')
display(des_segments_electrolyzer_dropdown)

Dropdown(description='des_segments_electrolyzer:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=None)

You selected: 3


In [7]:
# Set the area in which the facility is located
frequency_dropdown = create_dropdown('frequency')
display(frequency_dropdown)

Dropdown(description='frequency:', options=('1h', '1D', '1M', '1Y'), value='1h')

In [8]:
#temporal block
if frequency_dropdown.value == '1h':
    temporal_block = 'hourly'
elif frequency_dropdown == '1D':
    temporal_block = 'daily'
elif frequency_dropdown == '1W':
    temporal_block = 'weekly'
elif frequency_dropdown == '1M':
    temporal_block = 'monthly'
elif frequency_dropdown == '1Q':
    temporal_block = 'quarterly'
else: 
    temporal_block = 'yearly'

In [9]:
#required reports
report_name = 'Report'
reports = ['unit_flow', 'connection_flow', 'node_state', 'total_costs', 'unit_flow_op', 'node_slack_neg', 'node_slack_pos']

In [10]:
#financial parameters for the calculations
#when using the district heating as a revenue stream, the price cap is the upper limit in Denmark
#by using this parameter this can be adjusted to start with other than the max value
share_of_dh_price_cap = 0.5
price_level_power = 1
power_price_variance = 1

In [11]:
#scenario
scenario = 'Base'

#model name
model_name = 'toy'

#stochastics
stochastic_scenario = "realisation"
stochastic_structure = "deterministic"

In [16]:
#define parameters for the use of the roll forward (rolling horizon) approach
#set the number of slices within the modelling horizon
roll_forward_use = True
num_slices = 12
# Create DatetimeIndex for the range of dates
start_date = pd.Timestamp(str(year_dropdown.value) + '-01-01 00:00:00')
end_date = pd.Timestamp(str(year_dropdown.value) + '-12-31 23:00:00')
datetime_index = pd.date_range(start=start_date, end=end_date, freq=frequency_dropdown.value)
# Calculate the number of steps within the horizon
num_steps = len(datetime_index)
# Check if the number of slices can be used
# Find the largest integer divisor that fulfils the condition
for i in range(num_slices, 0, -1):
    if num_steps % i == 0:
        roll_forward_size = num_steps // i
        used_slices = i
        break
else:
    print("Cannot divide the number of steps into any integer slices. Please choose a different number of slices.")
    exit()
# Check if num_slices matches the used_slices
if num_slices != used_slices:
    print("\033[91mWARNING:\033[0m The specified number of slices (", num_slices, ")",
          "does not match the final division factor (", used_slices, ").",
         "\n The calculation uses the factor: ",used_slices, ".")


# Print warning message in red
print("\033[91mWARNING:\033[0m Please control if all the parameters are set correctly")

### Dictionary:

In [17]:
base_parameters = {
    'year': year_dropdown.value,
    'start_date': (pd.Timestamp(str(year_dropdown.value) + '-01-01 00:00:00')).strftime('%Y-%m-%d %H:%M:%S'),
    'end_date': (pd.Timestamp(str(year_dropdown.value) + '-12-31 23:00:00')).strftime('%Y-%m-%d %H:%M:%S'),
    'area': area_dropdown.value,
    'product': product_dropdown.value,
    'scenario': scenario,
    'frequency': frequency_dropdown.value,
    'model_name': model_name,
    'temporal_block': temporal_block,
    'stochastic_scenario': stochastic_scenario,
    'stochastic_structure': stochastic_structure,
    'report_name': report_name,
    'reports': reports,
    'electrolyzer_type': electrolyzer_type_dropdown.value,
    'des_segments_electrolyzer': des_segments_electrolyzer_dropdown.value,
    'share_of_dh_price_cap': share_of_dh_price_cap,
    'price_level_power': price_level_power,
    'power_price_variance': power_price_variance,
    'roll_forward_use': roll_forward_use,
    'roll_forward_size': roll_forward_size,
    'num_slices': num_slices,
    'num_steps': num_steps
}

### Execute standard routine:

In [ ]:
pm.execute_notebook(
    'data_preparation_nord_h2ub_spine.ipynb',
    'please_ignore.ipynb',
    parameters=base_parameters
)